<center>
<h1> TP-Projet d'optimisation numérique </h1>
<h1> Algorithme des Régions de Confiance </h1>
</center>

# Régions de confiance avec Pas de Cauchy 

## Implémentation 

1. Coder l'algorithme du pas de Cauchy d’un sous-problème de
régions de confiance (fichier `Pas_De_Cauchy.jl`). La spécification de cet algorithme est donnée ci-dessous.

In [41]:
using LinearAlgebra
using Documenter
using Markdown  
include("Pas_De_Cauchy.jl")
@doc Pas_De_Cauchy

#### Objet

Cette fonction calcule une solution approchée du problème

$$
\min_{||s||< \Delta} s^{t}g + \frac{1}{2}s^{t}Hs
$$

par le calcul du pas de Cauchy.

#### Syntaxe

```julia
s, e = Pas_De_Cauchy(g,H,delta)
```

#### Entrées

  * g : (Array{Float,1}) un vecteur de $\mathbb{R}^n$
  * H : (Array{Float,2}) une matrice symétrique de $\mathbb{R}^{n\times n}$
  * delta  : (Float) le rayon de la région de confiance

#### Sorties

  * s : (Array{Float,1}) une approximation de la solution du sous-problème
  * e : (Integer) indice indiquant l'état de sortie:      si g != 0          si on ne sature pas la boule            e <- 1          sinon            e <- -1      sinon          e <- 0

#### Exemple d'appel

```julia
g = [0; 0]
H = [7 0 ; 0 2]
delta = 1
s, e = Pas_De_Cauchy(g,H,delta)
```


2. Ecrire des tests exhaustifs (qui testent tous les cas de figure possibles) pour votre algorithme du Pas de Cauchy. Vous créerez pour cela un fichier `tester_pas_de_Cauchy.jl` dans le répertoire `test` sur le modèle des autres fichiers de tests et vous exécuterez dans la cellule de code ci-après ces tests.

In [42]:
using Test

# Tolérance pour les tests d'égalité
tol_erreur = sqrt(eps())

## ajouter les fonctions de test
include("../test/tester_pas_de_cauchy.jl")
include("../src/Pas_De_Cauchy.jl")

affiche = false

@testset "Test pas de cauchy" begin
	# Tester le pas de Cauchy
	tester_pas_de_cauchy(affiche,Pas_De_Cauchy)
end;

Test Summary:      | Pass  Total  Time
Test pas de cauchy |   10     10  1.2s


3. Coder l'algorithme des Régions de Confiance (fichier `Regions_De_Confiance.jl`). Sa spécification est donnée ci-dessous.

In [43]:
include("Regions_De_Confiance.jl")
@doc Regions_De_Confiance

#### Objet

Minimise une fonction de $\mathbb{R}^{n}$ à valeurs dans $\mathbb{R}$ en utilisant l'algorithme des régions de confiance. 

La solution approchées des sous-problèmes quadratiques est calculé  par le pas de Cauchy ou le pas issu de l'algorithme du gradient conjugue tronqué

#### Syntaxe

```julia
xmin, fxmin, flag, nb_iters = Regions_De_Confiance(algo,f,gradf,hessf,x0,option)
```

#### Entrées :

  * algo        : (String) string indicant la méthode à utiliser pour calculer le pas

      * "gct"   : pour l'algorithme du gradient conjugué tronqué
      * "cauchy": pour le pas de Cauchy
  * f           : (Function) la fonction à minimiser
  * gradf       : (Function) le gradient de la fonction f
  * hessf       : (Function) la hessiene de la fonction à minimiser
  * x0          : (Array{Float,1}) point de départ
  * options     : (Array{Float,1})

      * deltaMax       : utile pour les m-à-j de la région de confiance                $R_{k}=\left\{x_{k}+s ;\|s\| \leq \Delta_{k}\right\}$
      * gamma1, gamma2 : $0 < \gamma_{1} < 1 < \gamma_{2}$ pour les m-à-j de $R_{k}$
      * eta1, eta2     : $0 < \eta_{1} < \eta_{2} < 1$ pour les m-à-j de $R_{k}$
      * delta0         : le rayon de départ de la région de confiance
      * max_iter       : le nombre maximale d'iterations
      * Tol_abs        : la tolérence absolue
      * Tol_rel        : la tolérence relative
      * epsilon       : epsilon pour les tests de stagnation

#### Sorties:

  * xmin    : (Array{Float,1}) une approximation de la solution du problème :            $\min_{x \in \mathbb{R}^{n}} f(x)$
  * fxmin   : (Float) $f(x_{min})$
  * flag    : (Integer) un entier indiquant le critère sur lequel le programme s'est arrêté (en respectant cet ordre de priorité si plusieurs critères sont vérifiés)

      * 0    : CN1
      * 1    : stagnation du $x$
      * 2    : stagnation du $f$
      * 3    : nombre maximal d'itération dépassé
  * nb_iters : (Integer)le nombre d'iteration qu'à fait le programme

#### Exemple d'appel

```julia
algo="gct"
f(x)=100*(x[2]-x[1]^2)^2+(1-x[1])^2
gradf(x)=[-400*x[1]*(x[2]-x[1]^2)-2*(1-x[1]) ; 200*(x[2]-x[1]^2)]
hessf(x)=[-400*(x[2]-3*x[1]^2)+2  -400*x[1];-400*x[1]  200]
x0 = [1; 0]
options = []
xmin, fxmin, flag, nb_iters = Regions_De_Confiance(algo,f,gradf,hessf,x0,options)
```


4. Vérifier que les tests ci-dessous passent.

In [44]:
using Test

# Tolérance pour les tests d'égalité
tol_erreur = sqrt(eps())

## ajouter les fonctions de test
include("../test/fonctions_de_tests.jl")
include("../test/tester_regions_de_confiance.jl")
include("../src/Pas_De_Cauchy.jl")
include("../src/Regions_De_Confiance.jl")

affiche = false

@testset "Test rc avec cauchy" begin
	tester_regions_de_confiance(affiche,Regions_De_Confiance)
end;

iters = 864
Test Summary:       | Pass  Total  Time
Test rc avec cauchy |   30     30  2.9s


## Interprétation 

<!-- Pour ces questions, des représentations graphiques sont attendues pour corroborer vos réponses. -->

1. Soit $$ f_{1} : \mathbf{R}^3 \rightarrow \mathbf{R}$$ $$ (x_1,x_2, x_3) \mapsto  2 (x_1 +x_2 + x_3 -3)^2 + (x_1-x_2)^2 + (x_2 - x_3)^2$$ Quelle relation lie la fonction $f_1$ et son modèle de Taylor à l’ordre 2 ? Comparer alors les performances de Newton et RC-Pas de Cauchy sur cette fonction.

2.  Le rayon initial de la région de confiance est un paramètre important dans l’analyse
de la performance de l’algorithme. Sur quel(s) autre(s) paramètre(s) peut-on jouer
pour essayer d’améliorer cette performance ? Étudier l’influence d’au moins deux de
ces paramètres. Pour cela vous ferez des tests numériques et donnerez les résultats sous forme de tableaux et de graphiques.

## Réponses

1. $f_1$ est une fonction quadratique et est donc égale à son modèle de Taylor à l'ordre 2,

##### Comparaison des performances de Newton et RC-Pas de Cauchy sur $f_1$

In [55]:
include("Algorithme_De_Newton.jl")
include("Regions_De_Confiance.jl")
include("Pas_De_Cauchy.jl")

# Affichage les sorties de l'algorithme des Régions de confiance
function my_afficher_resultats(algo,nom_fct,point_init,xmin,fxmin,flag,sol_exacte,nbiters)
	println("-------------------------------------------------------------------------")
	printstyled("Résultats de : ",algo, " appliqué à ",nom_fct, " au point initial ", point_init, ":\n",bold=true,color=:blue)
	println("  * xsol = ",xmin)
	println("  * f(xsol) = ",fxmin)
	println("  * nb_iters = ",nbiters)
	println("  * flag = ",flag)
	println("  * sol_exacte : ", sol_exacte)
end

f1(x) = 2(x[1]+x[2]+x[3] -3)^2 + (x[1]-x[2])^2 + (x[2]-x[3])^2
grad_f1(x) = [4(x[1]+x[2]+x[3]-3)+2(x[1]-x[2]) ;
                        4(x[1]+x[2]+x[3]-3)-2(x[1]-x[2])+2(x[2]-x[3]) ;
                        4(x[1]+x[2]+x[3]-3)-2(x[2]-x[3])]

hess_f1(x) = [[6 2 4] ;     
              [2 8 2] ;     
              [4 2 6]]

x0 = [1,0,0]
options = []
sol_exacte = [1.0,1.0,1.0]

xmin,f_min,flag,nb_iters = Algorithme_De_Newton(f1,grad_f1,hess_f1,x0,options)
my_afficher_resultats("Newton","f1",x0,xmin,f_min,flag,sol_exacte,nb_iters)

xmin, fxmin, flag, nb_iters = Regions_De_Confiance("cauchy",f1,grad_f1,hess_f1,x0,options)
my_afficher_resultats("RC-Pas de Cauchy","f1",x0,xmin,f_min,flag,sol_exacte,nb_iters)

-------------------------------------------------------------------------
Résultats de : Newton appliqué à f1 au point initial [1, 0, 0]:
  * xsol = [1.0, 1.0, 0.9999999999999999]
  * f(xsol) = 1.232595164407831e-32
  * nb_iters = 1
  * flag = 0
  * sol_exacte : [1.0, 1.0, 1.0]
-------------------------------------------------------------------------
Résultats de : RC-Pas de Cauchy appliqué à f1 au point initial [1, 0, 0]:
  * xsol = [1.0000072386475123, 0.999999018224437, 0.9999907978013619]
  * f(xsol) = 1.232595164407831e-32
  * nb_iters = 32
  * flag = 2
  * sol_exacte : [1.0, 1.0, 1.0]


In [56]:
x1 = [10; 3; -2.2]

xmin,f_min,flag,nb_iters = Algorithme_De_Newton(f1,grad_f1,hess_f1,x1,options)
my_afficher_resultats("Newton","f1",x1,xmin,f_min,flag,sol_exacte,nb_iters)

xmin, fxmin, flag, nb_iters = Regions_De_Confiance("cauchy",f1,grad_f1,hess_f1,x1,options)
my_afficher_resultats("RC-Pas de Cauchy","f1",x1,xmin,f_min,flag,sol_exacte,nb_iters)

-------------------------------------------------------------------------
Résultats de : Newton appliqué à f1 au point initial [10.0, 3.0, -2.2]:
  * xsol = [1.0, 0.9999999999999996, 0.9999999999999987]
  * f(xsol) = 7.296963373294359e-30
  * nb_iters = 1
  * flag = 0
  * sol_exacte : [1.0, 1.0, 1.0]
-------------------------------------------------------------------------
Résultats de : RC-Pas de Cauchy appliqué à f1 au point initial [10.0, 3.0, -2.2]:
  * xsol = [1.0000087106761566, 0.9999993178172312, 0.9999899249573803]
  * f(xsol) = 7.296963373294359e-30
  * nb_iters = 1000
  * flag = 3
  * sol_exacte : [1.0, 1.0, 1.0]


On remarque que l'algorithme de Newton converge en 1 itération à chaque fois alors que l'algorithme RC-Pas de Cauchy converge en 32 et 1000 (trop) itérations. De plus, on peut également voir que l'algorithme de Newton converge vers une solution bien plus précise que l'algorithme des Régions de Confiance avec le Pas de Cauchy.

2. 

# Régions de confiance avec Gradient Conjugué
## Implémentation 

1. Implémenter l’algorithme du Gradient Conjugué Tronqué (fichier `Gradient_Conjugue_Tronque.jl`). Sa spécification est donnée ci-dessous.

In [25]:
include("Gradient_Conjugue_Tronque.jl")
@doc Gradient_Conjugue_Tronque

#### Objet

Cette fonction calcule une solution approchée du problème

$$
\min_{||s||< \Delta}  q(s) = s^{t} g + \frac{1}{2} s^{t}Hs
$$

par l'algorithme du gradient conjugué tronqué

#### Syntaxe

```julia
s = Gradient_Conjugue_Tronque(g,H,option)
```

#### Entrées :

  * g : (Array{Float,1}) un vecteur de $\mathbb{R}^n$
  * H : (Array{Float,2}) une matrice symétrique de $\mathbb{R}^{n\times n}$
  * options          : (Array{Float,1})

      * delta    : le rayon de la région de confiance
      * max_iter : le nombre maximal d'iterations
      * tol      : la tolérance pour la condition d'arrêt sur le gradient

#### Sorties:

  * s : (Array{Float,1}) le pas s qui approche la solution du problème : $min_{||s||< \Delta} q(s)$

#### Exemple d'appel:

```julia
gradf(x)=[-400*x[1]*(x[2]-x[1]^2)-2*(1-x[1]) ; 200*(x[2]-x[1]^2)]
hessf(x)=[-400*(x[2]-3*x[1]^2)+2  -400*x[1];-400*x[1]  200]
xk = [1; 0]
options = []
s = Gradient_Conjugue_Tronque(gradf(xk),hessf(xk),options)
```


2. Vérifier que les tests ci-dessous passent.

In [26]:
using Test

# Tolérance pour les tests d'égalité
tol_erreur = sqrt(eps())

## ajouter les fonctions de test
include("../test/fonctions_de_tests.jl")
include("../test/tester_gct.jl")
include("../src/Gradient_Conjugue_Tronque.jl")

affiche = false

@testset "Test gct" begin
	tester_gct(affiche,Gradient_Conjugue_Tronque)
end;

Test Summary: | Pass  Total  Time
Test gct      |    9      9  4.4s


3. Intégrer l’algorithme du Gradient Conjugué Tronqué dans le code de régions de confiance (fichier `Regions_De_Confiance.jl`).

4. Décommenter les tests avec le gradient conjugué dans `tester_regions_de_confiance.jl` et vérifier que les tests passent.

In [27]:
using Test

# Tolérance pour les tests d'égalité
tol_erreur = sqrt(eps())

## ajouter les fonctions de test
include("../test/fonctions_de_tests.jl")
include("../test/tester_regions_de_confiance.jl")
include("../src/Pas_De_Cauchy.jl")
include("../src/Gradient_Conjugue_Tronque.jl")
include("../src/Regions_De_Confiance.jl")

affiche = false

@testset "Test rc avec cauchy et gct" begin
	tester_regions_de_confiance(affiche,Regions_De_Confiance)
end;

iters = 864
Test Summary:              | Pass  Total  Time
Test rc avec cauchy et gct |   30     30  4.3s


## Interprétation  

1. Comparer la décroissance obtenue avec celle du pas de Cauchy, en imposant la sortie
dans l’algorithme au bout d’une itération seulement. Vous donnerez ci-après des résultats numériques. 
    1. Que remarquez vous ?
    2. Comparer la décroissance obtenue avec celle du pas de Cauchy dans le cas général.

3. Quels sont les avantages et inconvénients des deux approches ?

## Réponses

1. Comparaison de la décroissance obtenue avec celle du pas de Cauchy :

A. En imposant la sortie dans l'algorithme au bout d'une itération seulement, on remarque qu'on obtient une décroissance plus faible qu'avec le pas de Cauchy. En effet, le pas de Cauchy utilise une formule permettant de trouver le pas optimal pour le sous-problème de la région de confiance, tandis que l'algorithme du gradient conjugué tronqué utilise une direction de descente obtenue en résolvant un système linéaire tronqué, qui n'est pas forcément la meilleure direction de descente possible.

B. Dans le cas général, le Pas de Cauchy donne une décroissance plus forte que l'algorithme du gradient conjugué tronqué, car il utilise une méthode analytique pour trouver le pas optimal, tandis que l'algorithme du gradient conjugué tronqué se base sur des approximations linéaires pour déterminer la direction de descente.

2. Avantages et inconvénients des deux approches :
 - Pas de Cauchy : 
     * décroissance plus forte et plus rapide
     * plus coûteux en termes de temps de calcul (calcul d'une matrice hessienne à chaque itération)
     
 - Gradient Conjugué Tronqué :
     * décroissance moins forte et moins rapide
     * moins coûteux en termes de temps de calcul (ne nécessite que des produits matrice-vecteur)